# Quantum error correction - bit-flip code quantum kata

The **"Quantum error correction - bit-flip code"** quantum kata is a series of exercises designed to get you familiar with quantum error correction (QEC) and programming in Q#.  It introduces you to the simplest of QEC codes - the three-qubit bit-flip code, which encodes each logical qubit in three physical qubits and protects against single bit-flip error (equivalent to applying an X gate).  In practice quantum systems can have other types of errors, which will be considered in the following katas on quantum error correction.  

Each task is wrapped in one operation preceded by the description of the task.
Your goal is to fill in the blank (marked with // ... comment)
with some Q# code that solves the task. To verify your answer, run the cell using Ctrl+Enter (⌘+Enter on macOS).

The tasks are given in approximate order of increasing difficulty.

### Task 1. Parity Measurements

**Input:** three qubits (stored as an array of length 3) in an unknown basis state or in a superposition of basis states of the same parity.

**Output:** the parity of this state encoded as a value of `Result` type: `Zero` for parity 0 and `One` for parity 1.  The parity of basis state $| x_{0} x_{1} x_{2}\rangle$ is defined as $(x_{0} \otimes x_{1} \otimes x_{2})$.  After applying the operation the state of the qubits should not change. You can use exactly one measurement.
    
**Example:** $|000 \rangle$, $|101\rangle$ and $|011\rangle$ all have parity 0, while $|010\rangle$ and $|111\rangle$ have parity 1.

In [1]:
%kata T01_MeasureParity 

operation MeasureParity (register : Qubit[]) : Result {
    return Measure([PauliZ, PauliZ, PauliZ], register);
}

Testing on |000⟩ + |011⟩ with parity 0
Testing on |000⟩ + |101⟩ with parity 0
Testing on |000⟩ + |110⟩ with parity 0
Testing on |001⟩ + |010⟩ with parity 1
Testing on |001⟩ + |100⟩ with parity 1
Testing on |001⟩ + |111⟩ with parity 1
Testing on |010⟩ + |100⟩ with parity 1
Testing on |010⟩ + |111⟩ with parity 1
Testing on |011⟩ + |101⟩ with parity 0
Testing on |011⟩ + |110⟩ with parity 0
Testing on |100⟩ + |111⟩ with parity 1
Testing on |101⟩ + |110⟩ with parity 0


Success!

### Task 2. Encoding Codewords

**Input**: three qubits in the state $| \psi \rangle \otimes |00\rangle$, where $|\psi\rangle = \alpha |0\rangle + \beta |1\rangle$ is the state of the first qubit, i.e., `register[0]`.

**Goal**:  create a state $|\bar{\psi}\rangle := \alpha |000\rangle + \beta |111\rangle$ on these qubits.   

In [2]:
%kata T02_Encode 

operation Encode (register : Qubit[]) : Unit {
    CNOT(register[0], register[1]);
    CCNOT(register[0], register[1], register[2]);
}

Success!

### Task 3. Error Detection I

**Inputs:** three qubits that are either in the state $|\bar{\psi}\rangle = \alpha |000\rangle + \beta |111\rangle$ or in the state $X\mathbb{11} |\bar{\psi}\rangle = \alpha |100\rangle + \beta |011\rangle$.  

> Note that the second state is the first state with X applied to the first qubit, which corresponds to an X error happening on the first qubit.

**Output:** `Zero` if the input is $|\bar{\psi}\rangle$ (state without the error), `One` if the input is $X\mathbb{11} |\bar{\psi}\rangle$ (state with the error).  After applying the operation the state of the qubits should not change.

In [4]:
%kata T03_DetectErrorOnLeftQubit 

operation DetectErrorOnLeftQubit (register : Qubit[]) : Result {
    return Measure([PauliZ, PauliZ], register[0..1]);
}

Success!

### Task 4. Error Correction I

**Input:** three qubits that are either in the state $|\bar{\psi}\rangle = \alpha |000\rangle + \beta |111\rangle$ or in the state $X\mathbb{11} |\bar{\psi}\rangle = \alpha |100\rangle + \beta |011\rangle$.

**Goal:**  make sure that the qubits are returned to the state $|\bar{\psi}\rangle$  (i.e., determine whether an X error has occurred, and if so, fix it).

<br/>
<details>
  <summary><b>Need a hint? Click here</b> </summary>
  You can use task 3 to figure out which state you are given.
</details>

In [5]:
%kata T04_CorrectErrorOnLeftQubit

operation CorrectErrorOnLeftQubit (register : Qubit[]) : Unit {
    if DetectErrorOnLeftQubit(register) == One {
        X(register[0]);
    }
}

Success!

### Task 5. Error Detection II

**Input:** three qubits that are either in the state $|\bar{\psi}\rangle = \alpha |000\rangle + \beta |111\rangle$ or in one of the states $X\mathbb{11} |\bar{\psi}\rangle$, $\mathbb{1}X\mathbb{1} |\bar{\psi}\rangle$ or $\mathbb{11}X |\bar{\psi}\rangle$ (i.e., state $|\bar{\psi}\rangle$ with an X error applied to one of the qubits).

**Goal:** determine whether an X error has occurred, and if so, on which qubit.

| Error                     | Output |
|---------------------------|--------|
| None                      | 0      |
| $X\mathbb{11}$            | 1      |
| $\mathbb{1}X\mathbb{1}$   | 2      |
| $\mathbb{11}X$            | 3      |

After applying the operation the state of the qubits should not change.

In [8]:
%kata T05_DetectErrorOnAnyQubit

operation DetectErrorOnAnyQubit (register : Qubit[]) : Int {
    let e0 = Measure([PauliZ, PauliZ], register[0..1]);
    let e1 = Measure([PauliZ, PauliZ], register[1...]);

    if e0 == Zero {
        if e1 == Zero {
            return 0;
        }
        else {
            return 3;
        }
    }
    else {
        if e1 == Zero {
            return 1;
        }
        else {
            return 2;
        }
    }
    
}

Testing with no error.
Testing with error on qubit 1.
Testing with error on qubit 2.
Testing with error on qubit 3.


Success!

### Task 6. Error Correction II

**Input:** three qubits that are either in the state $|\bar{\psi}\rangle = \alpha |000\rangle + \beta |111\rangle$ or in one of the states $X\mathbb{11} |\bar{\psi}\rangle$, $\mathbb{1}X\mathbb{1} |\bar{\psi}\rangle$ or $\mathbb{11}X |\bar{\psi}\rangle$ (i.e., the qubits start in state $|\bar{\psi}\rangle$ with an X error possibly applied to one of the qubits).

**Goal:**  make sure that the qubits are returned to the state $|\bar{\psi}\rangle$ (i.e., determine whether an X error has occurred on any qubit, and if so, fix it).

In [9]:
%kata T06_CorrectErrorOnAnyQubit

operation CorrectErrorOnAnyQubit (register : Qubit[]) : Unit {
    let error = DetectErrorOnAnyQubit(register);

    if error > 0 {
        X(register[error-1]);
    }
}

Task 06: Testing on [PauliI,PauliI,PauliI]...
Task 06: Testing on [PauliX,PauliI,PauliI]...
Task 06: Testing on [PauliI,PauliX,PauliI]...
Task 06: Testing on [PauliI,PauliI,PauliX]...


Success!

> All the tasks in this kata have been dealing with X errors on single qubit.  The bit-flip code doesn't allow one to detect or correct a Z error or multiple X errors.  Indeed, 
  * A Z error on a logical state $|\psi\rangle = \alpha |0\rangle + \beta |1\rangle$ encoded using the bit-flip code would convert the encoded state $|\bar{\psi}\rangle = \alpha |000\rangle + \beta |111\rangle$ into $\alpha |000\rangle - \beta |111\rangle$,  which is a correct code word for logical state $\alpha |0\rangle - \beta |1\rangle$.  
  * Two X errors (say, on qubits 1 and 2) would convert $|\bar{\psi}\rangle$ to $\alpha |110\rangle + \beta |001\rangle$,  which is a code word for logical state $\beta |0\rangle + \alpha |1\rangle$ with one X error on qubit 3.

### Task 7. Logical X Gate

**Input:** three qubits that are either in the state $|\bar{\psi}\rangle = \alpha |000\rangle + \beta |111\rangle$ or in one of the states $X\mathbb{11} |\bar{\psi}\rangle$, $\mathbb{1}X\mathbb{1} |\bar{\psi}\rangle$ or $\mathbb{11}X |\bar{\psi}\rangle$ (i.e., state $|\bar{\psi}\rangle$ with an X error applied to one of the qubits).

**Goal:** apply a logical X operator, i.e., convert the qubits to the state $\bar{X} |\bar{\psi}\rangle = \beta |000\rangle + \alpha |111\rangle$ or one of the states that can be represented as $\bar{X} |\bar{\psi}\rangle$ with an X error applied to one of the qubits (for example, $\beta |010\rangle + \alpha |101\rangle$).  If the state has an error, you can fix it, but this is not necessary.

In [11]:
%kata T07_LogicalX

operation LogicalX (register : Qubit[]) : Unit {
    ApplyToEach(X, register);
}

Task 07: Testing on [PauliI,PauliI,PauliI]...
Task 07: Testing on [PauliX,PauliI,PauliI]...
Task 07: Testing on [PauliI,PauliX,PauliI]...
Task 07: Testing on [PauliI,PauliI,PauliX]...


Success!

### Task 8. Logical Z Gate

**Input:** three qubits that are either in the state $|\bar{\psi}\rangle = \alpha |000\rangle + \beta |111\rangle$ or in one of the states $X\mathbb{11} |\bar{\psi}\rangle$, $\mathbb{1}X\mathbb{1} |\bar{\psi}\rangle$ or $\mathbb{11}X |\bar{\psi}\rangle$ (i.e., state $|\bar{\psi}\rangle$ with an X error applied to one of the qubits).

**Goal:** apply a logical Z operator, i.e., convert the qubits to the state $\bar{Z} \bar{\psi}\rangle = \alpha |000\rangle - \beta |111\rangle$ or one of the states that can be represented as $\bar{Z} |\bar{\psi}\rangle$ with an X error applied to one of the qubits (for example, $\alpha |010\rangle - \beta |101\rangle$ ). If the state has an error, you can fix it, but this is not necessary.

In [12]:
%kata T08_LogicalZ

operation LogicalZ (register : Qubit[]) : Unit {
    ApplyToEach(Z, register);
}

Task 08: Testing on [PauliI,PauliI,PauliI]...
Task 08: Testing on [PauliX,PauliI,PauliI]...
Task 08: Testing on [PauliI,PauliX,PauliI]...
Task 08: Testing on [PauliI,PauliI,PauliX]...


Success!